In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
import torch.nn.functional as F
import math

from torch.optim import lr_scheduler

In [3]:
import argparse
import torch.optim as optim
from torchvision import datasets, transforms

In [4]:

# MNIST Dataset
train_dataset = dsets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

In [5]:
def squash(x):
    lengths2 = x.pow(2).sum(dim=2)
    lengths = lengths2.sqrt()
    x = x * (lengths2 / (1 + lengths2) / lengths).view(x.size(0), x.size(1), 1)
    return x

In [6]:
class AgreementRouting(nn.Module):
    def __init__(self, input_caps, output_caps, n_iterations):
        super(AgreementRouting, self).__init__()
        self.n_iterations = n_iterations
        self.b = nn.Parameter(torch.zeros((input_caps, output_caps)))

    def forward(self, u_predict):
        batch_size, input_caps, output_caps, output_dim = u_predict.size()

        c = F.softmax(self.b)
        s = (c.unsqueeze(2) * u_predict).sum(dim=1)
        v = squash(s)

        if self.n_iterations > 0:
            b_batch = self.b.expand((batch_size, input_caps, output_caps))
            for r in range(self.n_iterations):
                v = v.unsqueeze(1)
                b_batch = b_batch + (u_predict * v).sum(-1)

                c = F.softmax(b_batch.view(-1, output_caps)).view(-1, input_caps, output_caps, 1)
                s = (c * u_predict).sum(dim=1)
                v = squash(s)

        return v

In [7]:
class Encode_layer(nn.Module):
    def __init__(self, input_dim, output_dim, output_caps):
        super(Encode_layer, self).__init__()
        
        self.layer = nn.Linear(input_dim, output_dim*output_caps)
        self.output_dim = output_dim
        self.output_caps = output_caps
        
    def forward(self, input) : 
        output = self.layer(input)
        output = F.relu(output)
        output = output.view(output.shape[0], self.output_caps, self.output_dim)
        output = squash(output)
        
        return output

In [8]:
class Neuron_layer(nn.Module):
    def __init__(self, input_neurons, input_dim, output_dim, output_neurons, routing):
        super(Neuron_layer, self).__init__()
        self.input_neurons = input_neurons
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.output_neurons = output_neurons
        self.routing = routing
        self.weights = nn.Parameter(torch.Tensor(input_neurons, input_dim, output_neurons * output_dim))
        self.reset_parameters()
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.input_neurons)
        self.weights.data.uniform_(-stdv, stdv)
        
    def forward(self, input):
        
        input = input.unsqueeze(2)
        predict = input.matmul(self.weights)
        predict = predict.view(predict.size(0), self.input_neurons, self.output_neurons, self.output_dim)
        v = self.routing(predict)
        
        return v
        

In [9]:
class Learning(nn.Module) :
    def __init__(self, routing_iterations) :
        super(Learning, self).__init__()
        
        self.network1 = Encode_layer(784, 128, 4)
        
        routing_module1 = AgreementRouting(4, 3, routing_iterations)
        
        self.network2 = Neuron_layer(4, 128, 32, 3, routing_module1)
        
        routing_module2 = AgreementRouting(3, 10, routing_iterations)
        
        self.network3 = Neuron_layer(3, 32, 16, 10, routing_module2)
        
    def forward(self, input):
        
        output1 = self.network1(input)
        output2 = self.network2(output1)
        output3 = self.network3(output2)
        
        probs = output3.pow(2).sum(dim=2).sqrt()
        
        return probs

In [10]:
model=Learning(3)

In [11]:
class MarginLoss(nn.Module):
    def __init__(self, m_pos, m_neg, lambda_):
        super(MarginLoss, self).__init__()
        self.m_pos = m_pos
        self.m_neg = m_neg
        self.lambda_ = lambda_

    def forward(self, lengths, targets, size_average=True):
        t = torch.zeros(lengths.size()).long()
        if targets.is_cuda:
            t = t.cuda()
        t = t.scatter_(1, targets.data.view(-1, 1), 1)
        targets = Variable(t)
        losses = targets.float() * F.relu(self.m_pos - lengths).pow(2) + \
                 self.lambda_ * (1. - targets.float()) * F.relu(lengths - self.m_neg).pow(2)
        return losses.mean() if size_average else losses.sum()


In [12]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=15, min_lr=1e-6)

loss_fn = MarginLoss(0.9, 0.1, 0.5)

In [13]:
def train(epoch):
    model.train()
    correct=0.
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data.view(data.shape[0], -1)), Variable(target, requires_grad=False)
        optimizer.zero_grad()
        probs= model(data)
        
        loss = loss_fn(probs, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.data[0]))
            
        pred = probs.data.max(1, keepdim=True)[1]  # get the index of the max probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    print('Accuracy : {:.2f}%'.format(100. * correct / len(train_loader.dataset)))

In [14]:
import numpy as np

In [15]:
for epoch in range(100) :
    print(epoch)
    train(epoch)  #More stable 

0
Train Epoch: 0 [0/60000 (0%)]	Loss: 0.079742


/home/suii/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.
/home/suii/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [1000/60000 (2%)]	Loss: 0.075803
Train Epoch: 0 [2000/60000 (3%)]	Loss: 0.071463
Train Epoch: 0 [3000/60000 (5%)]	Loss: 0.065941
Train Epoch: 0 [4000/60000 (7%)]	Loss: 0.063325
Train Epoch: 0 [5000/60000 (8%)]	Loss: 0.056114
Train Epoch: 0 [6000/60000 (10%)]	Loss: 0.044342
Train Epoch: 0 [7000/60000 (12%)]	Loss: 0.044127
Train Epoch: 0 [8000/60000 (13%)]	Loss: 0.035468
Train Epoch: 0 [9000/60000 (15%)]	Loss: 0.035392
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.026028
Train Epoch: 0 [11000/60000 (18%)]	Loss: 0.026232
Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.019336
Train Epoch: 0 [13000/60000 (22%)]	Loss: 0.017153
Train Epoch: 0 [14000/60000 (23%)]	Loss: 0.015767
Train Epoch: 0 [15000/60000 (25%)]	Loss: 0.014884
Train Epoch: 0 [16000/60000 (27%)]	Loss: 0.013585
Train Epoch: 0 [17000/60000 (28%)]	Loss: 0.012300
Train Epoch: 0 [18000/60000 (30%)]	Loss: 0.015937
Train Epoch: 0 [19000/60000 (32%)]	Loss: 0.013069
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.013889
Train Epoch: 0

Train Epoch: 2 [46000/60000 (77%)]	Loss: 0.003626
Train Epoch: 2 [47000/60000 (78%)]	Loss: 0.001721
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.002002
Train Epoch: 2 [49000/60000 (82%)]	Loss: 0.003807
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.001028
Train Epoch: 2 [51000/60000 (85%)]	Loss: 0.001920
Train Epoch: 2 [52000/60000 (87%)]	Loss: 0.002831
Train Epoch: 2 [53000/60000 (88%)]	Loss: 0.003301
Train Epoch: 2 [54000/60000 (90%)]	Loss: 0.001957
Train Epoch: 2 [55000/60000 (92%)]	Loss: 0.002011
Train Epoch: 2 [56000/60000 (93%)]	Loss: 0.003088
Train Epoch: 2 [57000/60000 (95%)]	Loss: 0.001318
Train Epoch: 2 [58000/60000 (97%)]	Loss: 0.005376
Train Epoch: 2 [59000/60000 (98%)]	Loss: 0.003626
Accuracy : 96.30%
3
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.001684
Train Epoch: 3 [1000/60000 (2%)]	Loss: 0.001954
Train Epoch: 3 [2000/60000 (3%)]	Loss: 0.002421
Train Epoch: 3 [3000/60000 (5%)]	Loss: 0.000501
Train Epoch: 3 [4000/60000 (7%)]	Loss: 0.002109
Train Epoch: 3 [5000/60000 (8%)]	Loss: 0.00

Train Epoch: 5 [30000/60000 (50%)]	Loss: 0.001944
Train Epoch: 5 [31000/60000 (52%)]	Loss: 0.000616
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.002057
Train Epoch: 5 [33000/60000 (55%)]	Loss: 0.002370
Train Epoch: 5 [34000/60000 (57%)]	Loss: 0.002675
Train Epoch: 5 [35000/60000 (58%)]	Loss: 0.003102
Train Epoch: 5 [36000/60000 (60%)]	Loss: 0.001664
Train Epoch: 5 [37000/60000 (62%)]	Loss: 0.002094
Train Epoch: 5 [38000/60000 (63%)]	Loss: 0.001760
Train Epoch: 5 [39000/60000 (65%)]	Loss: 0.000932
Train Epoch: 5 [40000/60000 (67%)]	Loss: 0.003203
Train Epoch: 5 [41000/60000 (68%)]	Loss: 0.000808
Train Epoch: 5 [42000/60000 (70%)]	Loss: 0.003532
Train Epoch: 5 [43000/60000 (72%)]	Loss: 0.000498
Train Epoch: 5 [44000/60000 (73%)]	Loss: 0.000509
Train Epoch: 5 [45000/60000 (75%)]	Loss: 0.001097
Train Epoch: 5 [46000/60000 (77%)]	Loss: 0.000928
Train Epoch: 5 [47000/60000 (78%)]	Loss: 0.000962
Train Epoch: 5 [48000/60000 (80%)]	Loss: 0.002484
Train Epoch: 5 [49000/60000 (82%)]	Loss: 0.001967


...

Train Epoch: 88 [46000/60000 (77%)]	Loss: 0.000000
Train Epoch: 88 [47000/60000 (78%)]	Loss: 0.000001
Train Epoch: 88 [48000/60000 (80%)]	Loss: 0.000000
Train Epoch: 88 [49000/60000 (82%)]	Loss: 0.000000
Train Epoch: 88 [50000/60000 (83%)]	Loss: 0.000000
Train Epoch: 88 [51000/60000 (85%)]	Loss: 0.000000
Train Epoch: 88 [52000/60000 (87%)]	Loss: 0.000000
Train Epoch: 88 [53000/60000 (88%)]	Loss: 0.000000
Train Epoch: 88 [54000/60000 (90%)]	Loss: 0.000000
Train Epoch: 88 [55000/60000 (92%)]	Loss: 0.000000
Train Epoch: 88 [56000/60000 (93%)]	Loss: 0.000000
Train Epoch: 88 [57000/60000 (95%)]	Loss: 0.000000
Train Epoch: 88 [58000/60000 (97%)]	Loss: 0.000001
Train Epoch: 88 [59000/60000 (98%)]	Loss: 0.000000
Accuracy : 100.00%
89
Train Epoch: 89 [0/60000 (0%)]	Loss: 0.000000
Train Epoch: 89 [1000/60000 (2%)]	Loss: 0.000000
Train Epoch: 89 [2000/60000 (3%)]	Loss: 0.000000
Train Epoch: 89 [3000/60000 (5%)]	Loss: 0.000000
Train Epoch: 89 [4000/60000 (7%)]	Loss: 0.000001
Train Epoch: 89 [5000/

Train Epoch: 91 [27000/60000 (45%)]	Loss: 0.000009
Train Epoch: 91 [28000/60000 (47%)]	Loss: 0.000373
Train Epoch: 91 [29000/60000 (48%)]	Loss: 0.000502
Train Epoch: 91 [30000/60000 (50%)]	Loss: 0.000422
Train Epoch: 91 [31000/60000 (52%)]	Loss: 0.000063
Train Epoch: 91 [32000/60000 (53%)]	Loss: 0.000011
Train Epoch: 91 [33000/60000 (55%)]	Loss: 0.000001
Train Epoch: 91 [34000/60000 (57%)]	Loss: 0.000003
Train Epoch: 91 [35000/60000 (58%)]	Loss: 0.000064
Train Epoch: 91 [36000/60000 (60%)]	Loss: 0.000477
Train Epoch: 91 [37000/60000 (62%)]	Loss: 0.000106
Train Epoch: 91 [38000/60000 (63%)]	Loss: 0.000112
Train Epoch: 91 [39000/60000 (65%)]	Loss: 0.000227
Train Epoch: 91 [40000/60000 (67%)]	Loss: 0.000077
Train Epoch: 91 [41000/60000 (68%)]	Loss: 0.000000
Train Epoch: 91 [42000/60000 (70%)]	Loss: 0.000144
Train Epoch: 91 [43000/60000 (72%)]	Loss: 0.000001
Train Epoch: 91 [44000/60000 (73%)]	Loss: 0.000001
Train Epoch: 91 [45000/60000 (75%)]	Loss: 0.000030
Train Epoch: 91 [46000/60000 (7

Train Epoch: 94 [8000/60000 (13%)]	Loss: 0.000000
Train Epoch: 94 [9000/60000 (15%)]	Loss: 0.000001
Train Epoch: 94 [10000/60000 (17%)]	Loss: 0.000000
Train Epoch: 94 [11000/60000 (18%)]	Loss: 0.000000
Train Epoch: 94 [12000/60000 (20%)]	Loss: 0.000000
Train Epoch: 94 [13000/60000 (22%)]	Loss: 0.000000
Train Epoch: 94 [14000/60000 (23%)]	Loss: 0.000000
Train Epoch: 94 [15000/60000 (25%)]	Loss: 0.000000
Train Epoch: 94 [16000/60000 (27%)]	Loss: 0.000000
Train Epoch: 94 [17000/60000 (28%)]	Loss: 0.000000
Train Epoch: 94 [18000/60000 (30%)]	Loss: 0.000000
Train Epoch: 94 [19000/60000 (32%)]	Loss: 0.000000
Train Epoch: 94 [20000/60000 (33%)]	Loss: 0.000000
Train Epoch: 94 [21000/60000 (35%)]	Loss: 0.000000
Train Epoch: 94 [22000/60000 (37%)]	Loss: 0.000000
Train Epoch: 94 [23000/60000 (38%)]	Loss: 0.000000
Train Epoch: 94 [24000/60000 (40%)]	Loss: 0.000000
Train Epoch: 94 [25000/60000 (42%)]	Loss: 0.000000
Train Epoch: 94 [26000/60000 (43%)]	Loss: 0.000000
Train Epoch: 94 [27000/60000 (45%

Train Epoch: 96 [49000/60000 (82%)]	Loss: 0.000000
Train Epoch: 96 [50000/60000 (83%)]	Loss: 0.000000
Train Epoch: 96 [51000/60000 (85%)]	Loss: 0.000000
Train Epoch: 96 [52000/60000 (87%)]	Loss: 0.000000
Train Epoch: 96 [53000/60000 (88%)]	Loss: 0.000000
Train Epoch: 96 [54000/60000 (90%)]	Loss: 0.000000
Train Epoch: 96 [55000/60000 (92%)]	Loss: 0.000000
Train Epoch: 96 [56000/60000 (93%)]	Loss: 0.000000
Train Epoch: 96 [57000/60000 (95%)]	Loss: 0.000000
Train Epoch: 96 [58000/60000 (97%)]	Loss: 0.000000
Train Epoch: 96 [59000/60000 (98%)]	Loss: 0.000000
Accuracy : 100.00%
97
Train Epoch: 97 [0/60000 (0%)]	Loss: 0.000000
Train Epoch: 97 [1000/60000 (2%)]	Loss: 0.000000
Train Epoch: 97 [2000/60000 (3%)]	Loss: 0.000000
Train Epoch: 97 [3000/60000 (5%)]	Loss: 0.000000
Train Epoch: 97 [4000/60000 (7%)]	Loss: 0.000000
Train Epoch: 97 [5000/60000 (8%)]	Loss: 0.000000
Train Epoch: 97 [6000/60000 (10%)]	Loss: 0.000000
Train Epoch: 97 [7000/60000 (12%)]	Loss: 0.000000
Train Epoch: 97 [8000/6000

Train Epoch: 99 [30000/60000 (50%)]	Loss: 0.000000
Train Epoch: 99 [31000/60000 (52%)]	Loss: 0.000000
Train Epoch: 99 [32000/60000 (53%)]	Loss: 0.000000
Train Epoch: 99 [33000/60000 (55%)]	Loss: 0.000000
Train Epoch: 99 [34000/60000 (57%)]	Loss: 0.000000
Train Epoch: 99 [35000/60000 (58%)]	Loss: 0.000000
Train Epoch: 99 [36000/60000 (60%)]	Loss: 0.000000
Train Epoch: 99 [37000/60000 (62%)]	Loss: 0.001192
Train Epoch: 99 [38000/60000 (63%)]	Loss: 0.000000
Train Epoch: 99 [39000/60000 (65%)]	Loss: 0.000000
Train Epoch: 99 [40000/60000 (67%)]	Loss: 0.000000
Train Epoch: 99 [41000/60000 (68%)]	Loss: 0.000000
Train Epoch: 99 [42000/60000 (70%)]	Loss: 0.000000
Train Epoch: 99 [43000/60000 (72%)]	Loss: 0.000000
Train Epoch: 99 [44000/60000 (73%)]	Loss: 0.000000
Train Epoch: 99 [45000/60000 (75%)]	Loss: 0.000000
Train Epoch: 99 [46000/60000 (77%)]	Loss: 0.000000
Train Epoch: 99 [47000/60000 (78%)]	Loss: 0.000000
Train Epoch: 99 [48000/60000 (80%)]	Loss: 0.000000
Train Epoch: 99 [49000/60000 (8

In [16]:
#Test
total=0
correct=0.
#batch size 다를 땐 테스트가 안 됌?
for images, labels in test_loader:
    images = Variable(images.view(images.shape[0], -1))
    outputs= model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))


/home/suii/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.
/home/suii/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test Accuracy of the model on the 10000 test images: 98 %


In [17]:
adv_test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=1, 
                                          shuffle=False)

In [18]:
dummy=1
s=0.
t=0.


for (x, y) in adv_test_loader :
    x, y = Variable(x, requires_grad=True), Variable(y, requires_grad=False)
    probs= model(x.view(x.shape[0], -1))
    y_pred = np.argmax(probs.data.numpy())
    loss = nn.CrossEntropyLoss()(probs, y)
    loss.backward()
    epsilon = 0.1 
    x_grad   = torch.sign(x.grad.data)
    x_adversarial = torch.clamp(x.data + epsilon * x_grad, 0, 1)
    adversarial_probs= model(Variable(x_adversarial).view(x.shape[0], -1))
    y_pred_adversarial = np.argmax(adversarial_probs.data.numpy())
    
    print("{0} th example ".format(dummy))
    print ("True value: "+ str(y.data.numpy()[0])+"\nPredicted value : "+str(y_pred)+ "\nAdversarial :" + str(y_pred_adversarial)+"\n" )
    
    dummy+=1
    
    if y.data.numpy()[0]!=y_pred :
        t+=1
    
    if y_pred!=y_pred_adversarial :
        s+=1
    if dummy==100:
        break
        
print("Accuracy of test_model : {0:.3f} , Adversarials : {1:.3f}".format((t/dummy)*100, (s/dummy)*100))

1 th example 
True value: 7
Predicted value : 7
Adversarial :3

2 th example 
True value: 2
Predicted value : 2
Adversarial :3

3 th example 
True value: 1
Predicted value : 1
Adversarial :8

4 th example 
True value: 0
Predicted value : 0
Adversarial :0

5 th example 
True value: 4
Predicted value : 4
Adversarial :9

6 th example 
True value: 1
Predicted value : 1
Adversarial :7

7 th example 
True value: 4
Predicted value : 4
Adversarial :8

8 th example 
True value: 9
Predicted value : 9
Adversarial :3

9 th example 
True value: 5
Predicted value : 5
Adversarial :4

10 th example 
True value: 9
Predicted value : 9
Adversarial :9

11 th example 
True value: 0
Predicted value : 0
Adversarial :2

12 th example 
True value: 6
Predicted value : 6
Adversarial :6

13 th example 
True value: 9
Predicted value : 9
Adversarial :7

14 th example 
True value: 0
Predicted value : 0
Adversarial :0

15 th example 
True value: 1
Predicted value : 1
Adversarial :8

16 th example 
True value: 5
Predi

/home/suii/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.
/home/suii/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


29 th example 
True value: 0
Predicted value : 0
Adversarial :0

30 th example 
True value: 1
Predicted value : 1
Adversarial :3

31 th example 
True value: 3
Predicted value : 3
Adversarial :3

32 th example 
True value: 1
Predicted value : 1
Adversarial :8

33 th example 
True value: 3
Predicted value : 3
Adversarial :3

34 th example 
True value: 4
Predicted value : 4
Adversarial :0

35 th example 
True value: 7
Predicted value : 7
Adversarial :3

36 th example 
True value: 2
Predicted value : 2
Adversarial :2

37 th example 
True value: 7
Predicted value : 7
Adversarial :2

38 th example 
True value: 1
Predicted value : 1
Adversarial :8

39 th example 
True value: 2
Predicted value : 2
Adversarial :3

40 th example 
True value: 1
Predicted value : 1
Adversarial :1

41 th example 
True value: 1
Predicted value : 1
Adversarial :8

42 th example 
True value: 7
Predicted value : 7
Adversarial :3

43 th example 
True value: 4
Predicted value : 4
Adversarial :9

44 th example 
True value